In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import os

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
links = [
"utama",
"nasional",
"politik-dup",
"ekonomi",
"dunia",
"sukan",
"akhbar-cina",
"pendapat",
"rencana",
"analisis"
]

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [3]:
def crawl(url):
    while True:
        try:
            r = requests.get(url,headers = headers)
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "lxml")

    block = soup.find('div',attrs = {"class":"td-ss-main-content"})
    
    if block is None:
        return

    for link in block.find_all('h3', attrs = {"class":"entry-title td-module-title"}):

        href = link.find('a').get('href')

        hrefs.append(href)
    

In [4]:
max_worker = 10


for t in links:
    
    hrefs = []
    print(t)
    url = f'https://malaysiagazette.com/category/{t}'
    print(url)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    pgno = [a_ for a_ in a if 'page' in a_]
    max_page = max([int(a_.split('page/')[1].replace('/','')) for a_ in pgno])
    
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}/page/{a}' for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            future.result()
    
    hrefs = list(set(hrefs))
    with open(f'malaysiagazette-{t}-link.json', 'a') as f:
        json.dump(hrefs, f)
    


politik-dup
https://malaysiagazette.com/category/politik-dup


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 203/203 [10:20<00:00,  3.06s/it]


ekonomi
https://malaysiagazette.com/category/ekonomi


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 132/132 [09:00<00:00,  4.10s/it]


dunia
https://malaysiagazette.com/category/dunia


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.57s/it]


pendapat
https://malaysiagazette.com/category/pendapat


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:26<00:00,  1.14s/it]


rencana
https://malaysiagazette.com/category/rencana


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.22s/it]


In [3]:
url = []

for t in links:
    print(t)
    with open(f'malaysiagazette-{t}-link.json') as fopen:
        href = json.load(fopen)
    url.extend(href)

utama
nasional
politik-dup
ekonomi
dunia
sukan
akhbar-cina
pendapat
rencana
analisis


In [4]:
len(url)

131923

In [10]:
 with open(f'malaysiagazette-link.json', 'a') as f:
        json.dump(hrefs, f)

error in link:https://malaysiagazette.com/2021/02/28/fdam-persoal-mengapa-mkn-masih-belum-beri-kelonggaran-kepada-industri-kreatif/
'NoneType' object has no attribute 'text'
error in link:https://malaysiagazette.com/2021/10/11/rakyat-perlu-tahu-kenapa-ph-tarik-balik-semakan-keputusan-pulau-batu-puteh/
'NoneType' object has no attribute 'text'
error in link:https://malaysiagazette.com/2021/05/30/agong-raja-permaisuri-ucap-selamat-menyambut-pesta-kaamatan/
'NoneType' object has no attribute 'text'
error in link:https://malaysiagazette.com/2022/09/26/manifesto-dap-lebih-pragmatik/
'NoneType' object has no attribute 'text'
error in link:https://malaysiagazette.com/2020/03/07/dua-parah-bas-bawa-rombongan-guru-terlibat-kemalangan/
'NoneType' object has no attribute 'text'
error in link:https://malaysiagazette.com/2020/04/26/persediaan-pelajar-penting-hadapi-norma-baharu/
'NoneType' object has no attribute 'text'
error in link:https://malaysiagazette.com/2019/05/06/harga-minyak-susut-2-2/
'No

In [20]:
hrefs = list(set(url))

In [21]:
len(hrefs)

130522

In [7]:
def process_url(x):
    
    while True:
        try:
            r = requests.get(x, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "lxml")

    try:
        headline = soup.find('h1', class_="entry-title").text
        h = soup.find('div', class_="td-post-content td-pb-padding-side")
        p = [p.text for p in h.find_all("p")]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    
    data = {'url': x, 'headline': headline, 'content': p}
    return data

In [24]:
max_workers = 5

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in not_yet_scrape]
    
    for future in tqdm(futures, total=len(not_yet_scrape)):
        result = future.result()
        if result:
            with open('malaysiagazette.jsonl', 'a') as f:
                json.dump(result, f)
                f.write('\n')

  0%|                                                                                                                                                                                                | 0/58 [00:00<?, ?it/s]

error in link:https://malaysiagazette.com/2018/10/14/gerakan-masih-relevan-lebih-ramai-masuk-dari-keluar-parti-mah-siew-keong/
'NoneType' object has no attribute 'text'
error in link:https://malaysiagazette.com/2018/09/08/proses-mengundi-di-prk-balakong-mula-8-pagi/
'NoneType' object has no attribute 'text'


  2%|███▏                                                                                                                                                                                    | 1/58 [00:02<02:07,  2.23s/it]

error in link:https://malaysiagazette.com/2018/09/08/calon-ph-seri-setia-minta-pengundi-keluar-awal/
'NoneType' object has no attribute 'text'
error in link:https://malaysiagazette.com/2017/03/02/15858/
'NoneType' object has no attribute 'text'


  3%|██████▎                                                                                                                                                                                 | 2/58 [00:03<01:36,  1.72s/it]

error in link:https://malaysiagazette.com/2016/12/18/1589/
'NoneType' object has no attribute 'text'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:36<00:00,  1.61it/s]
